## LAB<sup>7-2</sup> 논리합을 수행하는 퍼셉트론 만들기 (p.240)

**실습 목표**

퍼셉트론 모델을 이용하여 참(1)과 거짓(-1)의 값을 갖는 두 입력에 대해 논리합을 수행할 수 있도록 모델을 학습시켜 보라.

### 1. 넘파이를 임포트하고, 두 입력에 곱해질 가중치 W와 편향 b를 준비하고 모두 0으로 초기화하자. 퍼셉트론 학습은 학습률 η가 필요한데 learning_rate로 준비한다.

In [1]:
import numpy as np

W, b = np.array([0,0]), 0.0
learning_rate = 0.01

### 2. 활성화 함수는 0보다 크면 1, 0보다 작으면 -1, 0이면 0을 출력하는 계단 함수로 준비.

In [2]:
def activation(s):
  if s > 0:   return 1
  elif s < 0: return -1
  return 0

### 3. 퍼셉트론 모델의 출력은 w<sup>T</sup>x + b를 활성화 함수에 넘기는 아래 함수로 구현된다.

In [3]:
def out(x):
  return activation(W.dot(x) + b)

### 4. 학습은 (입력 데이터, 정답 레이블)의 쌍으로 제공되어야 한다. 입력 데이터로 얻은 출력과 정답 레이블이 일치하지 않을 경우 학습 규칙에 따라 가중치를 갱신한다. 두 개의 노드에 들어오는 입력값은 각각 x0, x1에, 정답 레이블은 target 매개변수에 넘겨주어 다음과 같은 훈련 수행 함수를 만들 수 있다.

In [4]:
def train(x0, x1, target):
  global W, b
  X = np.array([x0,x1])
  y = out(X)

  ### 예측이 맞으면 아무것도 하지 않음---------------------------------
  if target == y: return False        # 가중치가 변경되지 않았음을 반환
  ### 예측이 틀리면 학습 실시------------------------------------------
  print('가중치 수정전 target : {} y:{}, b:{}, W:{}'.format(target, y, b, W))
  W = W + learning_rate * X * target    # 입력x출력 비례하여 가중치 변경
  b = b + learning_rate * 1 * target    # 편향: 입력이 1이라고 볼 수 있음
  print('가중치 수정후 target : {} y:{}, b:{}, W:{}'.format(target, y, b, W))
  return True

### 5. 퍼셉트론의 현재 가중치에 따라 입력에 대한 예측을 수행하는 함수는 다음과 같이 구현할 수 있다.

In [5]:
def predict(inputs):
  outputs = []
  for x in inputs:
    outputs.append(out(x))
  return outputs

### 6. 입력과 정답을 제공하여 학습을 시키자. 아래는 두 입력 노드에 제공할 수 있는 네 종류르이 모든 가능한 입력 예시에 대해 논리합이 출력해야 하는 결과를 정답으로 제공하여 훈련을 시키는 코드이다. 학습 결과 한 번의 반복만으로 적절한 가중치를 찾았다.

In [6]:
adjusted = 0
for i in range(100):
  adjusted += train(-1,-1, -1)    # 훈련 데이터 1
  adjusted += train(-1, 1,  1)    # 훈련 데이터 2
  adjusted += train( 1,-1,  1)    # 훈련 데이터 3
  adjusted += train( 1, 1,  1)    # 훈련 데이터 4
  print("iteration ------------", i)
  if not adjusted: break    # 모든 훈련에 대해 가중치 변화 없으면 학습종료
  adjusted = 0

가중치 수정전 target : -1 y:0, b:0.0, W:[0 0]
가중치 수정후 target : -1 y:0, b:-0.01, W:[0.01 0.01]
가중치 수정전 target : 1 y:-1, b:-0.01, W:[0.01 0.01]
가중치 수정후 target : 1 y:-1, b:0.0, W:[0.   0.02]
가중치 수정전 target : 1 y:-1, b:0.0, W:[0.   0.02]
가중치 수정후 target : 1 y:-1, b:0.01, W:[0.01 0.01]
iteration ------------ 0
iteration ------------ 1


### 7. 네 개의 입력을 제공하고 제대로 논리합으로 동작하는지 확인해 보자.

In [7]:
X = [[-1,-1], [-1,1], [1,-1], [1,1]]
yhat = predict(X)
print('x0 x1  y')
for i in range(len(X)):
  print('{0:2d} {1:2d} {2:2d}'.format(X[i][0], X[i][1], yhat[i]))

x0 x1  y
-1 -1 -1
-1  1  1
 1 -1  1
 1  1  1


## LAB<sup>7-3</sup> 다양한 논리 연산이 가능하게 퍼셉트론 훈련하기 (p.242)


**실습 목표**

LAB<sup>7-2</sup>를 이용하여 다양한 논리 연산이 가능하도록 퍼셉트론을 학습시켜 보자.

### **논리곱<sup>conjunction</sup>(AND) 구현**

### 1. 논리곱을 구현해 보자. 논리곱을 수행하는 회로 소자를 AND 게이트라고 하고 아래와 같은 기호로 표시한다. 이것은 두 입력이 모두 참(1)일 때 결과가 참이 되므로 아래와 같이 훈련을 실시할 수 있다. 역시 한 번의 반복만으로 해를 찾는다.

$ y = x_0 \land x_1 $

In [8]:
adjusted = 0
for i in range(100):
  adjusted += train(-1,-1, -1)    # 훈련 데이터 1
  adjusted += train(-1, 1, -1)    # 훈련 데이터 2
  adjusted += train( 1,-1, -1)    # 훈련 데이터 3
  adjusted += train( 1, 1,  1)    # 훈련 데이터 4
  print("iteration ------------", i)
  if not adjusted: break    # 모든 훈련에 대해 가중치 변화 없으면 학습종료
  adjusted = 0

가중치 수정전 target : -1 y:1, b:0.01, W:[0.01 0.01]
가중치 수정후 target : -1 y:1, b:0.0, W:[0.02 0.  ]
가중치 수정전 target : -1 y:1, b:0.0, W:[0.02 0.  ]
가중치 수정후 target : -1 y:1, b:-0.01, W:[0.01 0.01]
iteration ------------ 0
iteration ------------ 1


### 2. 네 가지 입력에 대한 출력을 LAB<sup>7-2</sup>처럼 확인해 보자. 논리곱 동작을 확인할 수 있다.

In [9]:
X = [[-1,-1], [-1,1], [1,-1], [1,1]]
yhat = predict(X)
print('x0 x1  y')
for i in range(len(X)):
  print('{0:2d} {1:2d} {2:2d}'.format(X[i][0], X[i][1], yhat[i]))

x0 x1  y
-1 -1 -1
-1  1 -1
 1 -1 -1
 1  1  1


### **NAND 구현**

### 1. NAND 연산을 수행하는 논리 게이트는 아래와 같은 기호로 표시되는데, 두 입력의 논리곱을 부정한 것이다.
$ y = \neg (x_0 \land x_1) $

NAND 연산이 중요한 이유는 어떠한 불리언 연산이라도 이 NAND 연산의 조합으로 만들 수 있기 때문이다. 우리의 퍼셉트론도 이 기능을 수행할 수 있게 만들어 보자. 이 연산은 논리곱의 결과를 부정negation한 것이다. 따라서 훈련 단계를 아래와 같이 수정하면 된다.

In [10]:
adjusted = 0
for i in range(100):
    adjusted += train(-1,-1,  1)    # 훈련 데이터 1
    adjusted += train(-1, 1,  1)    # 훈련 데이터 2
    adjusted += train( 1,-1,  1)    # 훈련 데이터 3
    adjusted += train( 1, 1, -1)    # 훈련 데이터 4
    print("iteration -------------", i)
    if not adjusted: break  # 모든 훈련에 대해 가중치 변화 없으면 학습종료
    adjusted = 0 

가중치 수정전 target : 1 y:-1, b:-0.01, W:[0.01 0.01]
가중치 수정후 target : 1 y:-1, b:0.0, W:[0. 0.]
가중치 수정전 target : 1 y:0, b:0.0, W:[0. 0.]
가중치 수정후 target : 1 y:0, b:0.01, W:[-0.01  0.01]
가중치 수정전 target : 1 y:-1, b:0.01, W:[-0.01  0.01]
가중치 수정후 target : 1 y:-1, b:0.02, W:[0. 0.]
가중치 수정전 target : -1 y:1, b:0.02, W:[0. 0.]
가중치 수정후 target : -1 y:1, b:0.01, W:[-0.01 -0.01]
iteration ------------- 0
iteration ------------- 1


### 2. 네 가지 입력에 대한 출력은 NAND 연산의 결과일 것이다.

In [11]:
X = [[-1,-1], [-1,1], [1,-1], [1,1]]
yhat = predict(X)
print('x0 x1  y')
for i in range(len(X)):
  print('{0:2d} {1:2d} {2:2d}'.format(X[i][0], X[i][1], yhat[i]))

x0 x1  y
-1 -1  1
-1  1  1
 1 -1  1
 1  1 -1


### 도전문제 7.1 XOR의 진리표를 이용하여 퍼셉트론을 학습시켜 보자
XOR는 아래와 같은 기호로 표시된다. 진리표가 그 아래 나타나 있다. 이런 논리 연산을 수행하는 퍼셉트론을 만들 수 있을까? 만들어지지 않는다면 그 이유를 생각해 보라.
$ y = (x_0 \land \neg x_1) \lor (\neg x_0 \land x_1) $

|x0|x1| y|
|-:|-:|-:|
|-1|-1|-1|
|-1| 1| 1|
| 1|-1| 1|
| 1| 1|-1|

In [14]:
adjusted = 0
for i in range(100):
    adjusted += train(-1,-1, -1)    # 훈련 데이터 1
    adjusted += train(-1, 1,  1)    # 훈련 데이터 2
    adjusted += train( 1,-1,  1)    # 훈련 데이터 3
    adjusted += train( 1, 1, -1)    # 훈련 데이터 4
    print("iteration -------------", i)
    if not adjusted: break  # 모든 훈련에 대해 가중치 변화 없으면 학습종료
    adjusted = 0 

가중치 수정전 target : -1 y:0, b:0.0, W:[0. 0.]
가중치 수정후 target : -1 y:0, b:-0.01, W:[0.01 0.01]
가중치 수정전 target : 1 y:-1, b:-0.01, W:[0.01 0.01]
가중치 수정후 target : 1 y:-1, b:0.0, W:[0.   0.02]
가중치 수정전 target : 1 y:-1, b:0.0, W:[0.   0.02]
가중치 수정후 target : 1 y:-1, b:0.01, W:[0.01 0.01]
가중치 수정전 target : -1 y:1, b:0.01, W:[0.01 0.01]
가중치 수정후 target : -1 y:1, b:0.0, W:[0. 0.]
iteration ------------- 0
가중치 수정전 target : -1 y:0, b:0.0, W:[0. 0.]
가중치 수정후 target : -1 y:0, b:-0.01, W:[0.01 0.01]
가중치 수정전 target : 1 y:-1, b:-0.01, W:[0.01 0.01]
가중치 수정후 target : 1 y:-1, b:0.0, W:[0.   0.02]
가중치 수정전 target : 1 y:-1, b:0.0, W:[0.   0.02]
가중치 수정후 target : 1 y:-1, b:0.01, W:[0.01 0.01]
가중치 수정전 target : -1 y:1, b:0.01, W:[0.01 0.01]
가중치 수정후 target : -1 y:1, b:0.0, W:[0. 0.]
iteration ------------- 1
가중치 수정전 target : -1 y:0, b:0.0, W:[0. 0.]
가중치 수정후 target : -1 y:0, b:-0.01, W:[0.01 0.01]
가중치 수정전 target : 1 y:-1, b:-0.01, W:[0.01 0.01]
가중치 수정후 target : 1 y:-1, b:0.0, W:[0.   0.02]
가중치 수정전 target : 1 y:-1, b:0.0, 